In [1]:
import os
import sys
import re
import json
import requests
import time
import math
import unicodedata
import numpy as np
import pandas as pd
from difflib import get_close_matches

sys.path.append('../')
pd.set_option('max_rows', 250)
pd.set_option('max_columns', 250)
pd.options.mode.chained_assignment = None

import util.input_output as io
import preprocessor.names as n
from preprocessor.data_loader import DataLoader
from util.paths import PATH_DATA_FORMATTED, PATH_DATA_RAW


stores_list_columns = [
    n.FIELD_CUSTOMER, 
    n.FIELD_STORE_NAME, 
    n.FIELD_CITY, 
    'zip_code', 
    'iris_code_commune',
]

In [2]:
def clean_city_field(serie: pd.Series, acronyms=None):

    # serie = serie.str.split('(', expand=True)[0]
    serie = serie.str.replace(r"\(.*\)", "")
    serie = serie.str.replace(r"(?: -).*", "")  # after ' -' should be removed same for /
    serie = serie.str.replace(r"(?: /).*", "")
    serie = serie.str.replace(r"(?:[0-9]).*", "")

    if acronyms is None:
        acronyms = []
    
    acronyms += [
        ('S/SAONE', 'SUR SAONE'),
        ('S/SEINE', 'SUR SEINE'),
        ('S/S', 'SOUS '), 
        ('S/', 'SUR '), 
        ('/', ' SUR '), 
        ('SAINT', 'ST'),
        (' FD', ' FERRAND'),
        (' EN PRO', ' EN PROVENCE')
    ]

    for old, new in acronyms:
        serie = serie.str.replace(old, new)

    acronyms = '|'.join(['-', '\'', '\$', 'ZAC ', 'NORD ', 'SUD ', 'CEDEX ', 'DRIVE '])
    serie = serie.str.replace(acronyms, ' ')
    
    acronyms = '|'.join(['Ã©', 'Ã¨'])
    serie = serie.str.replace(acronyms, 'E')
    
    serie = serie.str.replace(' +', ' ', regex=False)  # Remove double spaces
    return serie.str.strip()


def clean_carrefour_sell_out_data(tmp: pd.DataFrame):
    tmp['is_closed'] = tmp[n.FIELD_STORE_NAME].str.startswith('FER ')
    acronyms = '|'.join(['CARREFOUR ', 'CRF ', 'FER '])
    cities = tmp[n.FIELD_STORE_NAME].str.replace(acronyms, '')
    tmp[n.FIELD_CITY] = tmp[n.FIELD_CITY].fillna(cities)
    tmp.loc[:, n.FIELD_CITY] = clean_city_field(tmp[n.FIELD_CITY])
    tmp[n.FIELD_CITY] = tmp[n.FIELD_CITY].str.upper()
    return tmp


def clean_carrefour_market_sell_out_data(tmp: pd.DataFrame):
    tmp['is_closed'] = tmp[n.FIELD_STORE_NAME].str.startswith('FER ')
    acronyms = '|'.join(['MKT ', 'CMA ', 'FER '])
    cities = tmp[n.FIELD_STORE_NAME].str.replace(acronyms, '')

    tmp[n.FIELD_CITY] = tmp[n.FIELD_CITY].fillna(cities)
    tmp.loc[:, n.FIELD_CITY] = clean_city_field(tmp[n.FIELD_CITY])
    tmp[n.FIELD_CITY] = tmp[n.FIELD_CITY].str.upper()
    return tmp


def clean_intermarche_sell_out_data(tmp: pd.DataFrame):
    tmp[n.FIELD_CITY] = tmp[n.FIELD_CITY].fillna(tmp[n.FIELD_STORE_NAME])
    tmp.loc[:, n.FIELD_CITY] = clean_city_field(tmp[n.FIELD_CITY])
    tmp[n.FIELD_CITY] = tmp[n.FIELD_CITY].str.upper()
    return tmp


def clean_franprix_sell_out_data(tmp: pd.DataFrame):
    tmp.loc[:, n.FIELD_CITY] = clean_city_field(
        tmp[n.FIELD_CITY], acronyms=[('S/S', 'SUR SEINE '), ('/M ', 'SUR MARNE')])

    tmp[n.FIELD_CITY] = tmp[n.FIELD_CITY].str.replace('/', ' SUR ')
    tmp[n.FIELD_CITY] = tmp[n.FIELD_CITY].str.upper()

    # Stores are called FRANPRIX, FX, FPX (no need to clean because we already know the city)
    acronyms = '|'.join(['FRANPRIX ', 'LP ', 'SOCODIS - '])
    cities = tmp[n.FIELD_STORE_NAME].str.replace(acronyms, '')
    cities = clean_city_field(cities)

    tmp[n.FIELD_CITY] = tmp[n.FIELD_CITY].str.upper()
    tmp[n.FIELD_CITY] = tmp[n.FIELD_CITY].fillna(cities)
    return tmp


def clean_casino_sell_out_data(tmp: pd.DataFrame):
    cities = tmp[n.FIELD_STORE_NAME].copy()
    acronyms = '|'.join(['LPE ', 'PS ', 'SM CASINO ', 'GEANT ', 'LP '])
    cities = tmp[n.FIELD_STORE_NAME].str.replace(acronyms, '')

    # Remove everything that comes after the city name
    acronyms = '|'.join(
        ['COURS ', 'RUE ', 'PLACE ', 'HOTEL ', 'AV ', 'BD ', 'GARE ', 'MAIRIE ', 'PALAIS ',
         'PL ', 'RTE ', 'ALLEEE ', 'CH ', 'PCE ', 'GD ', 'ESPACE ', 'BLD ', 'DOCTEUR '])
    cities = cities.str.replace(acronyms, '- ')
    cities = clean_city_field(cities)
    
    tmp[n.FIELD_CITY] = tmp[n.FIELD_CITY].str.upper()
    tmp[n.FIELD_CITY] = tmp[n.FIELD_CITY].fillna(cities)
    return tmp


def clean_leaderprice_sell_out_data(tmp: pd.DataFrame):
    acronyms = '|'.join(['LP ', 'SOCODIS - '])
    cities = tmp[n.FIELD_STORE_NAME].str.replace(acronyms, '')
    cities = cities.str.upper()

    acronyms = '|'.join(['//'])
    cities = cities.str.replace(acronyms, ' - ')
    
    tmp[n.FIELD_CITY] = tmp[n.FIELD_CITY].fillna(cities)
    tmp[n.FIELD_CITY] = tmp[n.FIELD_CITY].str.upper()
    tmp[n.FIELD_CITY] = clean_city_field(tmp[n.FIELD_CITY])
    return tmp


def clean_stores_city_field():
    cleaning_function = {
        'CARREFOUR': clean_carrefour_sell_out_data,
        'CARREFOUR MARKET': clean_carrefour_market_sell_out_data,
        'GEANT - CASINO': clean_casino_sell_out_data,
        'FRANPRIX': clean_franprix_sell_out_data,
        'INTERMARCHE': clean_intermarche_sell_out_data,
        'LEADERPRICE': clean_leaderprice_sell_out_data,
    }

    frames = list()
    for customer in sell_out_data[n.FIELD_CUSTOMER].unique():
        frames.append(cleaning_function[customer](sell_out_data[sell_out_data[n.FIELD_CUSTOMER] == customer]))

    sell_out = pd.concat(frames, ignore_index=True, axis=0, sort=False)

    forced_match = [
        ('ST GERMAIN L HAYE', 'ST GERMAIN EN LAYE'),
        ('RUEIL MALM', 'RUEIL MALMAISON '),
        ('ROSNY SUR BOIS', 'ROSNY SOUS BOIS'),
        ('BOURG EN BR', 'BOURG EN BRESSE '),
        ('ND ', 'NOTRE DAME '),
        ('JUAN LES PINS', 'ANTIBES'),
        ('CLERMONT FD', 'CLERMONT FERRAND')
    ]
    for old, new in forced_match:
        sell_out[n.FIELD_CITY] = sell_out[n.FIELD_CITY].str.replace(old, new)
        
    return sell_out


def test_at_least_one_word_in_common(city, match):
    # words_re = re.compile("|".join([city]))
    # return words_re.search(match)
    gap_words = set(['PORT', 'ST', 'STE', 'STS', 'STES', 'BOURG', 'D', 'LE', 'L', 'SUR', 'LES', 'LA', 'EN'])
    city = set(city.split()).difference(gap_words)
    return city.intersection(match.split())


def find_best_match(cities_no_match):
    """ Look for cities for a close enough match """
    high_fuzzy_match = dict()
    high_fuzzy_match_check = dict()
    startwith_match = dict()
    startwith_match_check = dict()

    for city in list(cities_no_match):
        # words with high similarities
        match = get_close_matches(city, cities[n.FIELD_CITY], n=1, cutoff=0.92)  # Fuzzy string search
        if match:
            if test_at_least_one_word_in_common(city, match[0]):
                high_fuzzy_match.update({city: match[0]})
            else:
                if city.startswith(match[0]):
                    high_fuzzy_match_check.update({city: match[0]})
        else:
            match = sorted([x for x in cities[n.FIELD_CITY] if city.startswith(x)], reverse=True)
            if match and test_at_least_one_word_in_common(city, match[0]):
                startwith_match.update({city: match[0]})
            else:
                match = sorted([x for x in cities[n.FIELD_CITY] if x.startswith(city)], reverse=True)
                if not match:
                    match = sorted([x for x in cities[n.FIELD_CITY] if x.startswith(city)], reverse=True)

                if match and test_at_least_one_word_in_common(city, match[0]):
                    startwith_match_check.update({city: match[0]})
    return high_fuzzy_match, startwith_match, startwith_match_check, high_fuzzy_match_check


def standardize_name(serie):
    return (serie
            .str.replace('|'.join(['-', '\'']), ' ')
            .str.upper()
            .apply(lambda x: unicodedata.normalize('NFKD', x).encode('ASCII', 'ignore').decode())
            .str.replace('SAINT', 'ST')
           )

def create_city_key(serie):
    articles = '|'.join(['LE ', 'LA ', 'LES ', ' LES', ' LE', ' LA', ])
    serie = serie.str.replace(articles, '')
    return serie.str.split().str.join(sep='').str.strip()


def merge_with_official_city_table(df, cities_table, col_merge, cols_check):
    """ Function to keep the best match based on city name & zip code """
    df = (df.merge(cities_table, on=col_merge, how='left', suffixes=['_org', '']))
    
    df['filter'] = (df[cols_check[0]] == df[cols_check[1]]) | (df[cols_check[0]].isnull())
    df = (df
          .sort_values(['filter'], ascending=False)
          .drop_duplicates([n.FIELD_CUSTOMER, n.FIELD_STORE_NAME]))
    return df


def format_official_insee_city_list():
    city_location = (io.read_csv(PATH_DATA_RAW, 'insee', 'insee_data_cities_cleaned.csv', index_col=0)
                     .rename(columns={'codeinsee': 'iris_code_commune'})
                     .filter([n.FIELD_CITY, 'zip_code', 'latitude', 'longitude', 'iris_code_commune']))

    city_location[n.FIELD_CITY] = city_location[n.FIELD_CITY].str.replace('\\', ' ')
    city_location['key'] = standardize_name(city_location[n.FIELD_CITY])
    city_location['key'] = create_city_key(city_location['key'])
    
    for col in ['zip_code', 'iris_code_commune']:
        city_location[col] = city_location[col].astype(str).str.zfill(5)
    
    city_location['code'] = city_location['zip_code'].astype(str).str[:2]
    return city_location.drop_duplicates()


def format_official_insee_department_list():
    departments = (io.read_csv(PATH_DATA_RAW, 'insee', 'departments.csv', encoding='utf-8')
                   .filter(['code', 'name'])
                   .rename(columns={'name': 'countrySecondarySubdivision'}))
    departments['countrySecondarySubdivision'] = standardize_name(departments['countrySecondarySubdivision'])
    return departments


def find_exact_matches(sell_out, cities):
    
    all_cities = set(sell_out[n.FIELD_CITY])
    store_city_match = dict()
    store_city_match.update({city: city for city in all_cities.intersection(cities[n.FIELD_CITY])})
    store_city_match.update({
        ''.join(city.split()): city
        for city in cities[n.FIELD_CITY].unique()
        if ''.join(city.split()) in cities and len(city.split()) > 1
    })

    cities_no_match = all_cities.difference(set(store_city_match.keys()))    
    print(len(cities), len(set(store_city_match.keys())), len(cities_no_match))
    
    return store_city_match, cities_no_match

def aggregate_results_and_remove_ambiguous_matches(high_fuzzy_match, startwith_match, 
                                                   startwith_match_check, high_fuzzy_match_check,
                                                   store_city_match, cities_no_match, sell_out):
    for match in [high_fuzzy_match, startwith_match, startwith_match_check, high_fuzzy_match_check]:
        store_city_match.update(match)

    # Some city names are not explicit enough to provide an exact (non-ambiguous match so we will need to scrap them)
    count = cities.groupby(n.FIELD_CITY)['zip_code'].nunique()
    count = (count[count > 1]).index
    more_than_one_match = set(dict(filter(lambda x: x[1] in count, store_city_match.items())))

    all_cities = set(sell_out[n.FIELD_CITY])
    cities_no_match = all_cities.difference(set(store_city_match.keys()))
    cities_to_scrap = dict(filter(lambda x: x[0] in more_than_one_match, store_city_match.items()))
    store_city_match = dict(filter(lambda x: x[0] not in more_than_one_match, store_city_match.items()))
    print(len(all_cities), len(set(store_city_match.keys())), len(cities_no_match), len(cities_to_scrap.keys()))
    
    return store_city_match, cities_no_match, cities_to_scrap

In [3]:
def match_sell_out_cleaned_store_field(sell_out, cities):
    """ First get all cities with an exact match """
    store_city_match, cities_no_match = find_exact_matches(sell_out, cities)

    """ Look for cities for a close enough match """
    high_fuzzy_match, startwith_match, startwith_match_check, high_fuzzy_match_check = find_best_match(cities_no_match)
    store_city_match, cities_no_match, cities_to_scrap = aggregate_results_and_remove_ambiguous_matches(
        high_fuzzy_match, startwith_match, startwith_match_check, high_fuzzy_match_check, 
        store_city_match, cities_no_match, sell_out
    )

    """ Separate into two list (incl. list of all stores to scrap on the web) """
    stores_list = (sell_out.copy()
              .filter([n.FIELD_CUSTOMER, n.FIELD_STORE_NAME, n.FIELD_CITY]))

    stores_list = stores_list[stores_list[n.FIELD_CITY].isin(store_city_match)]  # only the one with a relevant match
    stores_list['key_merge'] = stores_list[n.FIELD_CITY].map(store_city_match)
    cities['key_merge'] = cities[n.FIELD_CITY]

    stores_list = (stores_list[stores_list['key_merge'].notnull()]
                   .merge(cities[['city', 'zip_code', 'key_merge', 'iris_code_commune', 'latitude', 'longitude']], 
                          on='key_merge', how='left', suffixes=['_org', '']))

    print(stores_list.shape, len(stores_list['city_org'].unique()))

    stores_list = stores_list.filter(stores_list_columns)

    no_stores = (sell_out.copy()
                 .filter([n.FIELD_CUSTOMER, n.FIELD_STORE_NAME, n.FIELD_CITY]))

    no_stores['key_several_match'] = no_stores[n.FIELD_CITY].map(cities_to_scrap)
    no_stores['key_several_match'] = no_stores['key_several_match'].where(
        ~no_stores[n.FIELD_CITY].isin(cities_no_match), '')

    no_stores = no_stores[no_stores['key_several_match'].notnull()]
    print(no_stores.shape, len(no_stores[n.FIELD_CITY].unique()))

    return stores_list, no_stores

In [9]:
""" Sell-out formatting """
file_name = 'sell_out_dts_clean.csv'
sell_out_data = (io.read_csv(PATH_DATA_FORMATTED, file_name, 
                             usecols=[n.FIELD_CUSTOMER, n.FIELD_STORE_NAME, n.FIELD_CITY])
                 .drop_duplicates([n.FIELD_CUSTOMER, n.FIELD_STORE_NAME]))
sell_out_data[n.FIELD_CUSTOMER] = sell_out_data[n.FIELD_CUSTOMER].str.upper()
sell_out = clean_stores_city_field()
print(sell_out[[n.FIELD_CUSTOMER, n.FIELD_STORE_NAME]].drop_duplicates().shape)
print(sell_out[[n.FIELD_CITY]].drop_duplicates().sort_values(n.FIELD_CITY).to_string());

""" List of matched stores & input web scrapper """
cities = format_official_insee_city_list()
stores_list, no_stores = match_sell_out_cleaned_store_field(sell_out, cities)
no_stores.to_csv('missing_match_store_v2.csv')
stores_list.to_csv('stores_list_1.csv')

/home/ubuntu/.conda/envs/carlsberg/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


(9098, 2)
                                          city
7184                                          
419                                  ABBEVILLE
8508                                   ABIDJAN
1085                                    ACCOUS
3568                                   ACHERES
886                         ACHEUX EN AMIENOIS
368                            ACHICOURT ARRAS
1555                                      AGDE
7502                     AGDE CHEMIN DES DUNES
6497                            AGDE DE GAULLE
3144                                    AGDE I
6226                          AGDE MONTESQUIEU
1133                                      AGEN
6778                            AGEN DE GAULLE
6157                    AGEN GENERAL DE GAULLE
6780                          AGEN MONTESQUIEU
5068                                   AGNEAUX
4478                                    AGNETZ
639                                       AHUN
7174                             AIGNAY LE DUC
142

36694 3049 1557
4606 3978 188 440
(8024, 9) 3978
(1074, 4) 628


/home/ubuntu/.conda/envs/carlsberg/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (0,1,6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [56]:
import unicodedata
def standardize_name(serie):
    return (serie
            .str.replace('|'.join(['-', '\'']), ' ')
            .str.upper()
            .apply(lambda x: unicodedata.normalize('NFKD', x).encode('ASCII', 'ignore').decode())
            .str.replace('SAINT', 'ST')
           )

def create_city_key(serie):
    articles = '|'.join(['LE ', 'LA ', 'LES ', ' LES', ' LE', ' LA', ])
    serie = serie.str.replace(articles, '')
    return serie.str.split().str.join(sep='').str.strip()

sell_out_data['city_clean'] = standardize_name(sell_out_data['city_clean'])
sell_out_data['key'] = create_city_key(sell_out_data['city_clean'])

In [61]:
def merge_with_official_city_table(df, cities_table, col_merge, cols_check):
    """ Function to keep the best match based on city name & zip code """
    df = (df.merge(cities_table, on=col_merge, how='left', suffixes=['_org', '']))
    
    df['filter'] = (df[cols_check[0]] == df[cols_check[1]]) | (df[cols_check[0]].isnull())
    df = (df
          .sort_values(['filter'], ascending=False)
          .drop_duplicates([n.FIELD_CUSTOMER, n.FIELD_STORE_NAME]))
    return df

In [113]:
tmp_matched.head()

,customer,store,city,zip_code,iris_code_commune
0,Leaderprice,BEAUREDIS,PARIS,75010,75056
582,Franprix,BAZAR MONDIAL,PARIS,75020,75056
571,Franprix,ALIMENTATION SUPER BOURET,PARIS,75019,75056
572,Franprix,FPX PARMENTIER ALIMENTAIRE,PARIS,75011,75056
573,Franprix,SOGIPOISSONNIERE,PARIS,75010,75056


In [ ]:
matched = 

In [13]:
""" WEB SCRAPPER """

' WEB SCRAPPER '

In [14]:
import pandas as pd
import numpy as np
import json
import requests
import time
import math

# Find geo_loc function
def get_geo_codes(address):
    key = '6j_USCas00Jqmpj2EFtKe18nLWPylj0gPTOLeBypt9U'
    apiversion = 1.0
    url = 'https://atlas.microsoft.com/search/fuzzy/json'
    parameters = {'Content-Type': 'application/json', 'subscription-key': key,
                  'api-version': apiversion, 'query': address}
    response = requests.get(url, parameters)

    return response.json()


def correct_location(j_son, country_code='FR'):
    res = j_son['results']
    found_french = False
    for i in range(len(res)):
        if res[i]['address']['countryCode'] == 'FR':
            found_french = True
            break
    if found_french:
        lat = res[i]['position']['lat']
        lon = res[i]['position']['lon']
        try:
            municipality = res[i]['address']['municipality']
        except:
            municipality = np.nan
        try:
            postal_code = res[i]['address']['postalCode']
        except:
            postal_code = np.nan
        try:
            countrySubdivision = res[i]['address']['countrySubdivision']
        except:
            countrySubdivision = np.nan
        try:
            countrySecondarySubdivision = res[i]['address']['countrySecondarySubdivision']
        except:
            countrySecondarySubdivision = np.nan
    if not found_french:
        lat = lon = municipality = postal_code = countrySubdivision = countrySecondarySubdivision = np.nan
    return lat, lon, municipality, postal_code, countrySubdivision, countrySecondarySubdivision


def scrap_addresses_from_web(missed_stores):
    # Runs scrapping with combined names
    start = time.time()
    result_addresses = pd.DataFrame()
    count = 0

    for index, row in missed_stores.iterrows():
        city_name = str(row['comb_name'])+", France"
        lat, lon, municipality, postal_code, countrySubdivision, countrySecondarySubdivision = correct_location(get_geo_codes(city_name))
        t = {'store': [row['store']], 'lat': [lat], 'lon': [lon], 'municipality': [municipality],
             'postal_code': [postal_code], 'countrySubdivision': [countrySubdivision], 'countrySecondarySubdivision': [countrySecondarySubdivision]}
        temp = pd.DataFrame(data=t)
        result_addresses = result_addresses.append(temp)
        count += 1 
        end = time.time()
        print("Web scrapping at : {0}% which took {1} seconds to run.".format(round(100*(count/len(missed_stores)),3),round(end - start)))

    missed_stores_results = pd.merge(missed_stores, result_addresses, on=["store"], how="left")
    return missed_stores_results


def get_indivual_scrapped_address(address):
    j_son = get_geo_codes(address + ', france')
    return j_son['results']
    

In [15]:
import os
import sys
import pandas as pd

sys.path.append('../')
pd.set_option('max_rows', 250)
pd.set_option('max_columns', 250)
pd.options.mode.chained_assignment = None

import util.input_output as io
import preprocessor.names as n
from util.paths import PATH_DATA_RAW, PATH_DATA_FORMATTED
import unicodedata


relevant_columns = [
    n.FIELD_CUSTOMER,
    n.FIELD_STORE_NAME,
    'zip_code',
    'lat_city',
    'lon_city',
    'city',
    'iris_code_commune'
]


def standardize_name(serie):
    return (serie
            .str.replace('|'.join(['-', '\'']), ' ')
            .str.upper()
            .apply(lambda x: unicodedata.normalize('NFKD', x).encode('ASCII', 'ignore').decode())
            .str.replace('SAINT', 'ST')
           )

def create_city_key(serie):
    articles = '|'.join(['LE ', 'LA ', 'LES ', ' LES', ' LE', ' LA', ])
    serie = serie.str.replace(articles, '')
    return serie.str.split().str.join(sep='').str.strip()


def merge_with_official_city_table(df, cities_table, col_merge, cols_check):
    """ Function to keep the best match based on city name & zip code """
    df = (df.merge(cities_table, on=col_merge, how='left', suffixes=['_org', '']))
    
    df['filter'] = (df[cols_check[0]] == df[cols_check[1]]) | (df[cols_check[0]].isnull())
    df = (df
          .sort_values(['filter'], ascending=False)
          .drop_duplicates([n.FIELD_CUSTOMER, n.FIELD_STORE_NAME]))
    return df


def format_official_insee_city_list():
    city_location = (io.read_csv(PATH_DATA_RAW, 'insee', 'insee_data_cities_cleaned.csv', index_col=0)
                     .rename(columns={'codeinsee': 'iris_code_commune'})
                     .filter([n.FIELD_CITY, 'zip_code', 'latitude', 'longitude', 'iris_code_commune']))

    city_location[n.FIELD_CITY] = city_location[n.FIELD_CITY].str.replace('\\', ' ')
    city_location['key'] = standardize_name(city_location[n.FIELD_CITY])
    city_location['key'] = create_city_key(city_location['key'])
    
    for col in ['zip_code', 'iris_code_commune']:
        city_location[col] = city_location[col].astype(str).str.zfill(5)
    
    city_location['code'] = city_location['zip_code'].astype(str).str[:2]
    return city_location.drop_duplicates()


def format_official_insee_department_list():
    departments = (io.read_csv(PATH_DATA_RAW, 'insee', 'departments.csv', encoding='utf-8')
                   .filter(['code', 'name'])
                   .rename(columns={'name': 'countrySecondarySubdivision'}))
    departments['countrySecondarySubdivision'] = standardize_name(departments['countrySecondarySubdivision'])
    return departments
    

def split_scrapper_output(missed_stores_results):
    relevant_columns = [
        'customer', 
        'store',
        'lat', 
        'lon',
        'municipality', 
        'postal_code',
        'countrySecondarySubdivision'
    ]

    missed_stores_results = (missed_stores_results
                             .filter(relevant_columns)
                             .rename(columns={'postal_code': 'zip_code', 'municipality': 'city'}))

    idx = missed_stores_results['zip_code'].isnull()
    missing_stores, matched_stores = missed_stores_results[idx], missed_stores_results[~idx]
    print(missing_stores.shape, matched_stores.shape)
    
    return matched_stores, missing_stores


def match_complete_records_with_insee_official_city_list(matched_stores, missing_stores, cities):
    """ Stores with known city & zip code fields """
    matched_stores[n.FIELD_CITY] = matched_stores[n.FIELD_CITY].fillna(
        matched_stores['zip_code'].map(cities[[n.FIELD_CITY, 'zip_code']].set_index('zip_code').to_dict()[n.FIELD_CITY]))

    matched_stores['key'] = standardize_name(matched_stores[n.FIELD_CITY])
    matched_stores['key'] = create_city_key(matched_stores['key'])
    matched_stores['code'] = matched_stores['zip_code'].astype(str).str[:2]
    print(matched_stores.shape)

    matched_stores = merge_with_official_city_table(matched_stores, cities, 'zip_code', ['key_org', 'key'])
    print(matched_stores.shape)

    idx = matched_stores[n.FIELD_CITY].isnull()
    tmp = matched_stores[idx]
    tmp = tmp.iloc[:, (~tmp.isnull().all(axis=0)).values]
    tmp.columns = [col.replace('_org', '') for col in tmp.columns]
    tmp = (merge_with_official_city_table(tmp, cities, 'key', ['code_org', 'code'])
           .filter(relevant_columns))

    matched_stores = pd.concat(
        [matched_stores[~idx].filter(relevant_columns), tmp[tmp[n.FIELD_CITY].notnull()]], axis=0, sort=False)
    missing_stores = (pd.concat([tmp[tmp[n.FIELD_CITY].isnull()], missing_stores], axis=0, sort=False)
                      .filter(missing_stores.columns))
    print(missing_stores.shape)

    unknown_stores = missing_stores[missing_stores['city'].isnull()]
    missing_stores = missing_stores[missing_stores['city'].notnull()]
    print(unknown_stores.shape)
    
    return matched_stores, missing_stores, unknown_stores 


def match_incomplete_records_with_insee_official_city_list(matched_stores, missing_stores, unknown_stores, cities):
    """ Stores with missing zip code field (or no match at previous step)"""
    for col in ['city', 'countrySecondarySubdivision']:
        missing_stores[col] = standardize_name(missing_stores[col].fillna(''))

    columns = [
        n.FIELD_CUSTOMER, 
        n.FIELD_STORE_NAME, 
        'lat_city', 
        'lon_city', 
        'city', 
        'code', 
        'countrySecondarySubdivision'
    ]

    missing_stores = (missing_stores
                      .merge(counties, on='countrySecondarySubdivision', how='left')
                      .filter(columns))

    missing_stores['city'] = standardize_name(missing_stores['city'])
    missing_stores['key'] = create_city_key(missing_stores['city'])
    missing_stores = (merge_with_official_city_table(missing_stores, cities, 'key', ('code_org', 'code')))
    tmp = missing_stores[missing_stores['filter'] == False]
    missing_stores = missing_stores[missing_stores['filter'] == True].filter(relevant_columns)

    potential_match = cities[['city', 'code', 'zip_code']]
    for index, row in tmp.iterrows():
        match = [x for x in potential_match['city'].unique() 
                 if row['city_org'].startswith(x) 
                 and set(row['city_org'].split()) & set(x.split())
                ]

        idx = potential_match['city'].isin(match)
        if row['code_org']:
            idx = idx & (potential_match['code'] == row['code_org'])
        match2 = potential_match[idx][['city', 'zip_code']].values

        if len(match2):
            tmp.loc[index, 'city'] = match2[0][0]
            tmp.loc[index, 'zip_code'] = match2[0][1] 
    #     print(row['city_org'], row['code_org'], match, match2)

    tmp = merge_with_official_city_table(
        tmp.filter(relevant_columns + ['code_org']), cities, 'zip_code', ('code_org', 'code'))

    matched_stores = (pd.concat([matched_stores, missing_stores, tmp[tmp[n.FIELD_CITY].notnull()]], 
                                axis=0, sort=False)
                      .filter(matched_stores.columns))
    
    unknown_stores = (pd.concat([unknown_stores, tmp[tmp[n.FIELD_CITY].isnull()]], sort=False)
                      .filter(unknown_stores.columns))
    print(unknown_stores.shape)
    
    return matched_stores, unknown_stores


def manually_match_incomplete_records_with_insee_official_city_list(matched_stores, unknown_stores, cities):
    """ Manual filling of few missing entries """
    manual_matches = [
        ('LA BARRE', 'DEUIL LA BARRE'), 
        ('JUAN LES PINS', 'ANTIBES'),
        ('MORET ', 'MORET SUR LOING'),
        ('LE GRAND BORNAND', 'LE GRAND BORNAND'),
        ('DISTRIJOLIE', 'MANTES LA JOLIE'),
        ('EIFFELDIS', 'ISSY LES MOULINEAUX')
        ()
    ]
    for old, new in manual_matches:
        unknown_stores['city'] = unknown_stores['city'].where(
            ~unknown_stores['store'].str.contains(old), new)

    unknown_stores = (unknown_stores
                      .filter([n.FIELD_CUSTOMER, n.FIELD_STORE_NAME, n.FIELD_CITY])
                      .merge(cities[[n.FIELD_CITY, 'zip_code', 'iris_code_commune']], on=n.FIELD_CITY, how='left'))

    matched_stores = pd.concat([matched_stores, unknown_stores[unknown_stores[n.FIELD_CITY].notnull()]], 
                               axis=0, sort=False)
    
    unknown_stores = unknown_stores[unknown_stores[n.FIELD_CITY].isnull()]
    print(unknown_stores.shape)
    
    return matched_stores, unknown_stores


In [16]:
""" Scrap location information from web """

file_name_to_scrap = 'missing_match_store_v2.csv'

missed_stores = pd.read_csv(file_name_to_scrap, encoding = 'ISO-8859-1')
missed_stores = missed_stores[['customer','store','city']]
missed_stores = missed_stores[~missed_stores['customer'].isnull()]
missed_stores['city'] = missed_stores['city'].fillna('')
missed_stores = missed_stores.drop_duplicates(subset=['customer','store','city'])

# Combines names
def unique(sequence):
    seen = set()
    return [x for x in sequence if not (x in seen or seen.add(x))]

missed_stores['comb_name'] = missed_stores[['customer','store', 'city']].apply(' '.join, axis=1).apply(
    lambda x: ' '.join(unique(x.split())))

missed_stores_results = scrap_addresses_from_web(missed_stores)
# missed_stores_results.to_csv('missing_match_web_scrap_comb_names.csv', index=False)

DEBUG 2019-03-06 14:08:58,528 Starting new HTTPS connection (1): atlas.microsoft.com:443
DEBUG 2019-03-06 14:09:04,109 https://atlas.microsoft.com:443 "GET /search/fuzzy/json?Content-Type=application%2Fjson&subscription-key=6j_USCas00Jqmpj2EFtKe18nLWPylj0gPTOLeBypt9U&api-version=1.0&query=CARREFOUR+BERCY+2%2C+France HTTP/1.1" 200 1121
DEBUG 2019-03-06 14:09:04,117 Starting new HTTPS connection (1): atlas.microsoft.com:443
DEBUG 2019-03-06 14:09:04,196 https://atlas.microsoft.com:443 "GET /search/fuzzy/json?Content-Type=application%2Fjson&subscription-key=6j_USCas00Jqmpj2EFtKe18nLWPylj0gPTOLeBypt9U&api-version=1.0&query=CARREFOUR+EVRY%2C+France HTTP/1.1" 200 3816
DEBUG 2019-03-06 14:09:04,204 Starting new HTTPS connection (1): atlas.microsoft.com:443
DEBUG 2019-03-06 14:09:04,288 https://atlas.microsoft.com:443 "GET /search/fuzzy/json?Content-Type=application%2Fjson&subscription-key=6j_USCas00Jqmpj2EFtKe18nLWPylj0gPTOLeBypt9U&api-version=1.0&query=CARREFOUR+LOMME%2C+France HTTP/1.1" 200

Web scrapping at : 0.093% which took 6 seconds to run.
Web scrapping at : 0.186% which took 6 seconds to run.
Web scrapping at : 0.279% which took 6 seconds to run.


DEBUG 2019-03-06 14:09:04,494 https://atlas.microsoft.com:443 "GET /search/fuzzy/json?Content-Type=application%2Fjson&subscription-key=6j_USCas00Jqmpj2EFtKe18nLWPylj0gPTOLeBypt9U&api-version=1.0&query=CARREFOUR+LA+PART+DIEU%28LYON%29+DIEU%2C+France HTTP/1.1" 200 3184
DEBUG 2019-03-06 14:09:04,502 Starting new HTTPS connection (1): atlas.microsoft.com:443


Web scrapping at : 0.372% which took 6 seconds to run.


DEBUG 2019-03-06 14:09:05,020 https://atlas.microsoft.com:443 "GET /search/fuzzy/json?Content-Type=application%2Fjson&subscription-key=6j_USCas00Jqmpj2EFtKe18nLWPylj0gPTOLeBypt9U&api-version=1.0&query=CARREFOUR+STE+GENEVIEVE+DES+BO%2C+France HTTP/1.1" 200 8170
DEBUG 2019-03-06 14:09:05,028 Starting new HTTPS connection (1): atlas.microsoft.com:443
DEBUG 2019-03-06 14:09:05,170 https://atlas.microsoft.com:443 "GET /search/fuzzy/json?Content-Type=application%2Fjson&subscription-key=6j_USCas00Jqmpj2EFtKe18nLWPylj0gPTOLeBypt9U&api-version=1.0&query=CARREFOUR+VITROLLES%2C+France HTTP/1.1" 200 8262
DEBUG 2019-03-06 14:09:05,178 Starting new HTTPS connection (1): atlas.microsoft.com:443


...

Web scrapping at : 99.534% which took 448 seconds to run.
Web scrapping at : 99.628% which took 448 seconds to run.


DEBUG 2019-03-06 14:16:26,467 https://atlas.microsoft.com:443 "GET /search/fuzzy/json?Content-Type=application%2Fjson&subscription-key=6j_USCas00Jqmpj2EFtKe18nLWPylj0gPTOLeBypt9U&api-version=1.0&query=LEADERPRICE+PRIMA+5+ST+DENIS%2C+France HTTP/1.1" 200 6626
DEBUG 2019-03-06 14:16:26,475 Starting new HTTPS connection (1): atlas.microsoft.com:443
DEBUG 2019-03-06 14:16:26,565 https://atlas.microsoft.com:443 "GET /search/fuzzy/json?Content-Type=application%2Fjson&subscription-key=6j_USCas00Jqmpj2EFtKe18nLWPylj0gPTOLeBypt9U&api-version=1.0&query=LEADERPRICE+MR+EDDAOUDI+ST+GRATIEN%2C+France HTTP/1.1" 200 5709
DEBUG 2019-03-06 14:16:26,573 Starting new HTTPS connection (1): atlas.microsoft.com:443


Web scrapping at : 99.721% which took 448 seconds to run.
Web scrapping at : 99.814% which took 448 seconds to run.


DEBUG 2019-03-06 14:16:26,908 https://atlas.microsoft.com:443 "GET /search/fuzzy/json?Content-Type=application%2Fjson&subscription-key=6j_USCas00Jqmpj2EFtKe18nLWPylj0gPTOLeBypt9U&api-version=1.0&query=LEADERPRICE+BLT+FRANCHEVILLE%2C+France HTTP/1.1" 200 6137
DEBUG 2019-03-06 14:16:26,917 Starting new HTTPS connection (1): atlas.microsoft.com:443
DEBUG 2019-03-06 14:16:27,006 https://atlas.microsoft.com:443 "GET /search/fuzzy/json?Content-Type=application%2Fjson&subscription-key=6j_USCas00Jqmpj2EFtKe18nLWPylj0gPTOLeBypt9U&api-version=1.0&query=LEADERPRICE+MID+MONTREUIL%2C+France HTTP/1.1" 200 8682


Web scrapping at : 99.907% which took 448 seconds to run.
Web scrapping at : 100.0% which took 448 seconds to run.


In [32]:
missed_stores_results[missed_stores_results['store'].str.contains('CHESSY')]

,customer,store,city,comb_name,lat,lon,municipality,postal_code,countrySubdivision,countrySecondarySubdivision
533,FRANPRIX,LPE CHESSY,CHESSY,FRANPRIX LPE CHESSY,45.88709,4.62227,Chessy,NaN,Auvergne-Rhône-Alpes,Rhône
534,FRANPRIX,LPE CHESSY,CHESSY,FRANPRIX LPE CHESSY,48.87888,2.76473,Marne-la-Vallée,77700,Île-de-France,"Val-de-Marne, Seine-et-Marne, Seine-Saint-Denis"
940,GEANT - CASINO,PS CHESSY LES MINES,CHESSY LES MINES,GEANT - CASINO PS CHESSY LES MINES,46.67705,4.37178,Montceau-les-Mines,71300,Bourgogne-Franche-Comté,Saône-et-Loire
1018,GEANT - CASINO,LPE CHESSY - PL. EDMOND CHARTIER,CHESSY,GEANT - CASINO LPE CHESSY PL. EDMOND CHARTIER,48.87907,2.76478,Marne-la-Vallée,77700,Île-de-France,"Val-de-Marne, Seine-et-Marne, Seine-Saint-Denis"
1100,LEADERPRICE,LPE CHESSY,CHESSY,LEADERPRICE LPE CHESSY,45.88709,4.62227,Chessy,NaN,Auvergne-Rhône-Alpes,Rhône
1101,LEADERPRICE,LPE CHESSY,CHESSY,LEADERPRICE LPE CHESSY,48.87888,2.76473,Marne-la-Vallée,77700,Île-de-France,"Val-de-Marne, Seine-et-Marne, Seine-Saint-Denis"


In [18]:
""" Check scrapper results and match with official insee list """

cities = format_official_insee_city_list()
counties = format_official_insee_department_list()

matched_stores, missing_stores = split_scrapper_output(missed_stores_results)

""" Step 1 - Stores with known city & zip code fields """
matched_stores, missing_stores, unknown_stores = match_complete_records_with_insee_official_city_list(
    matched_stores, missing_stores, cities)

""" Step 2 - Stores with missing zip code field (or no match at previous step)"""
matched_stores, unknown_stores = match_incomplete_records_with_insee_official_city_list(
    matched_stores, missing_stores, unknown_stores, cities)

""" Step 3 - Manual filling of few missing entries """
matched_stores, unknown_stores = manually_match_incomplete_records_with_insee_official_city_list(
    matched_stores, unknown_stores, cities)

matched_stores.to_csv('stores_list_2.csv')

(239, 7) (1171, 7)
(1171, 9)
(899, 16)
(244, 7)
(13, 7)
(15, 7)
(7, 5)


In [22]:
""" Consolidate results into one master list """

tmp1 = pd.read_csv('stores_list_1.csv')
tmp2 = pd.read_csv('stores_list_2.csv')
consolidated_stores_list = pd.concat([tmp1, tmp2], axis=0, sort=False)

""" create dcd specific store id """
consolidated_stores_list.sort_values([n.FIELD_CUSTOMER, n.FIELD_STORE_NAME])
consolidated_stores_list['dcd_store_id'] = (
    (consolidated_stores_list[n.FIELD_CUSTOMER] != consolidated_stores_list[n.FIELD_CUSTOMER].shift())
    |
    (consolidated_stores_list[n.FIELD_STORE_NAME] != consolidated_stores_list[n.FIELD_STORE_NAME].shift())
).cumsum().astype(str).str.zfill(4)

consolidated_stores_list.to_csv(os.path.join(PATH_DATA_FORMATTED, 'consolidated_stores_list.csv'))

print(consolidated_stores_list.shape)
print(consolidated_stores_list.groupby([n.FIELD_CUSTOMER, n.FIELD_STORE_NAME])['zip_code'].nunique().unique())

(9137, 7)
[1 2]


In [ ]:
consolidated_stores_list[~consolidated_stores_list['customer'].isin(['LEADERPRICE', 'FRANPRIX'])].to_csv(
    os.path.join(PATH_DATA_FORMATTED, 'consolidated_stores_list_scoped.csv'))

""" Franprix & Leaderprice already have city & zip_code informations """

In [ ]:
file_name = 'sell_out_dts_clean.csv'
sell_out_data = (io.read_csv(PATH_DATA_FORMATTED, file_name)
                 .drop_duplicates([n.FIELD_CUSTOMER, n.FIELD_STORE_NAME, 'zip_code', 'city']))

idx = ((sell_out_data['zip_code'].notnull()) | (sell_out_data['city'].notnull()))
print(sell_out_data[idx]['customer'].unique())
print(sell_out_data[idx].shape)
print(sell_out_data[sell_out_data['customer'].isin(['Franprix', 'Leaderprice'])].shape)

sell_out_data = (sell_out_data[sell_out_data['customer'].isin(['Franprix', 'Leaderprice'])]
                 .filter(['customer', 'store', 'city', 'zip_code', 'store_address']))
sell_out_data[n.FIELD_CUSTOMER] = sell_out_data[n.FIELD_CUSTOMER].str.upper()

sell_out_data['zip_code'] = (sell_out_data['zip_code']
                             .astype(str)
                             .str.strip('0')
                             .str.strip('.')
                             .str.zfill(5))

sell_out_data['city_clean'] = clean_city_field(sell_out_data['city'].str.upper())

missing = sell_out_data[sell_out_data[n.FIELD_CITY].isnull()]  # No information available
sell_out_data = sell_out_data[sell_out_data[n.FIELD_CITY].notnull()]

""" Merge on available zip_code """
tmp = merge_with_official_city_table(sell_out_data, cities, 'zip_code', ['key_org', 'key'])

idx = tmp['city'].isnull()
matched, not_matched = tmp[~idx], tmp[idx]
matched = matched.filter(stores_list_columns)

""" Merge on city filed (same departement) """
not_matched = (not_matched[['customer', 'store', 'city_org', 'zip_code', 'city_clean', 'key_org']]
               .rename(columns={'key_org': 'key'}))
not_matched['code'] = not_matched['zip_code'].str[:2]

tmp = merge_with_official_city_table(not_matched, cities, 'key', ['code_org', 'code'])
tmp_matched = (tmp[tmp['city'].notnull()]
               .drop(columns=['zip_code'])
               .rename(columns={'zip_code_org': 'zip_code'})
               .filter(stores_list_columns))
matched = matched = pd.concat([matched, tmp_matched], axis=0, sort=False)

""" Manual match and close match """
tmp_missing = tmp[tmp['city'].isnull()]
potential_match = cities[['city', 'code', 'zip_code']]
tmp_missing = (tmp_missing[['customer', 'store', 'city_org', 'zip_code_org', 'key', 'code_org']]
               .rename(columns={'zip_code_org': 'zip_code'}))

manual_match = [
    ('LA PLAINE ST DENIS', 'ST DENIS'),
    ('SUCY/BRIE', 'SUCY EN BRIE'),
    ('JUAN LES PINS', 'ANTIBES'),
    ('MONTFAVET', 'AVIGNON')
]
for old, new in manual_match:
    tmp_missing['city_org'] = tmp_missing['city_org'].where(
        ~(tmp_missing['city_org'] == old), new)

for index, row in tmp_missing.iterrows():
    match = [x for x in potential_match['city'].unique() 
             if row['city_org'].startswith(x) 
             and set(row['city_org'].split()) & set(x.split())
            ]

    idx = potential_match['city'].isin(match)
    if row['code_org']:
        idx = idx & (potential_match['code'] == row['code_org'])
    match2 = potential_match[idx][['city', 'zip_code']].values

    if len(match2):
        tmp_missing.loc[index, 'city'] = match2[0][0]
        tmp_missing.loc[index, 'zip_code'] = match2[0][1] 
        
tmp_missing = (tmp_missing[tmp_missing['city'].notnull()]
               .filter(stores_list_columns + list(['code_org'])))

tmp_missing = (merge_with_official_city_table(tmp_missing, cities, 'zip_code', ('code_org', 'code'))
               .filter(stores_list_columns))

""" All matched stores """
matched = pd.concat([matched, tmp_missing], axis=0, sort=False)

In [224]:
consolidated_stores_list = (pd.read_csv(os.path.join(PATH_DATA_FORMATTED, 'consolidated_stores_list_scoped.csv'))
                            .drop(columns=['dcd_store_id'])
                            .filter(stores_list_columns))

consolidated_stores_list = pd.concat([consolidated_stores_list, matched], axis=0, sort=False)
consolidated_stores_list['dcd_store_id'] = (
    (consolidated_stores_list[n.FIELD_CUSTOMER] != consolidated_stores_list[n.FIELD_CUSTOMER].shift())
    |
    (consolidated_stores_list[n.FIELD_STORE_NAME] != consolidated_stores_list[n.FIELD_STORE_NAME].shift())
).cumsum().astype(str).str.zfill(4)

consolidated_stores_list.to_csv(os.path.join(PATH_DATA_FORMATTED, 'consolidated_stores_list_scoped_clean.csv'))

In [ ]:


x = consolidated_stores_list.groupby([n.FIELD_CUSTOMER, n.FIELD_STORE_NAME])['zip_code'].nunique()

In [230]:
# LEADERPRICE & FRANPRIX ARE SHIT
# REST SEEMS FINE (CMA, CARREFOUR, GEANT-CASINO, INTERMARCHE)
x[x > 1]

Series([], Name: zip_code, dtype: int64)

In [ ]:
# BOUROUGHS FOR PARIS / LYON / MARSEILLE

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="scrapper", format_string="%s, FRANCE")


def flatten_dict(d):
    new_dict = dict()
    for k, v in d.items():
        if isinstance(v, dict):
            new_dict.update(flatten_dict(v))
        else:
            new_dict.update({k: v})
    return new_dict


def search_store(store):
    # exactly_one=True (returns only first result)
    output = geolocator.geocode(store, addressdetails=True, exactly_one=False, language='fr')
    print(output)
    if output is not None:
        output = dict(filter(lambda x: x[0] in ['lat', 'lon', 'address'], output[0].raw.items()))
        output.update({n.FIELD_STORE_NAME + '_org': store})

        return pd.DataFrame(flatten_dict(output), index=[0])

    return pd.DataFrame({n.FIELD_STORE_NAME + '_org': store}, index=[0])


# from functools import partial
# df['location'] = df['name'].apply(partial(geocode, language='de'))

import time

# store = 'CARREFOUR BREST SUPERMARKET'
no_stores['key'] = no_stores[[n.FIELD_CUSTOMER, n.FIELD_CITY]].apply(' '.join, axis=1).apply(
    lambda x: ' '.join(set(x.split()))
)

stores_data = list()
for store in sorted(no_stores['key'].unique()):
    time.sleep(0.2)
    stores_data.append(search_store(store))
    
pd.concat(stores_data, axis=0, sort=True)